# Importing the relevant packages

In [ ]:
import pandas as pd
from math import ceil, floor
# pip install geopy
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

## Loading in the cleaned data after forward elimination

In [16]:
df = pd.read_csv('cleaned_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150634 entries, 0 to 150633
Data columns (total 35 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   town                       150634 non-null  object 
 1   flat_type                  150634 non-null  object 
 2   storey_range               150634 non-null  object 
 3   floor_area_sqm             150634 non-null  float64
 4   lease_commence_date        150634 non-null  int64  
 5   resale_price               150634 non-null  float64
 6   Tranc_Year                 150634 non-null  int64  
 7   Tranc_Month                150634 non-null  int64  
 8   mid                        150634 non-null  int64  
 9   hdb_age                    150634 non-null  int64  
 10  max_floor_lvl              150634 non-null  int64  
 11  1room_sold                 150634 non-null  int64  
 12  2room_sold                 150634 non-null  int64  
 13  3room_sold                 15

C:\Users\Admin\AppData\Local\Temp\ipykernel_15164\4103037086.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cleaned_data.csv')


# Feature Engineering

age_at_sale: to derive the property's age at the point of the sale. It is different from hdb_age

In [4]:
df['age_at_sale'] = df['Tranc_Year'] - df['lease_commence_date']

floor_cat: to consolidate the columns of 'mid' and 'storey range' into one. showing just upper, middle and lower floors of the block

In [8]:
floor_cat = []
for row in range(0, df.shape[0]):
    if df['mid'][row] <= ceil(0.25 * df['max_floor_lvl'][row]):
        floor_cat.append('lower') 
    elif df['mid'][row] < floor(0.75 * df['max_floor_lvl'][row]):
        floor_cat.append('middle')
    elif df['mid'][row] >= floor(0.75 * df['max_floor_lvl'][row]):
        floor_cat.append('upper')
df['floor_cat'] = floor_cat

demand: it is the culmination of all the different 'sold' columns. However weightage was applied to make it a more realistic representation of the market.

In [9]:
# More weightage has been given to 3,4 and 5 rooms sold because they form the larger market share out of the rest of them
df['demand'] = (0.025*df['1room_sold'])+\
(0.05*df['2room_sold'])+\
(0.2*df['3room_sold'])+\
(0.3*df['4room_sold'])+\
(0.3*df['5room_sold'])+\
(0.05*df['exec_sold'])+\
(0.05*df['multigen_sold'])+\
(0.025*df['studio_apartment_sold'])

amenities_within_1km: to group the 'Malls_Within_1km' and the 'Hawkers_Within_1km' together

In [10]:
df['amenities_within_1km'] = df['Mall_Within_1km'] + df['Hawker_Within_1km']

mrt development: To give a ranking based on how much the development of an MRT station actually drives resale price.
There were 2 steps. First doing research and mapping out when each mrt station was built and then comparing them to the transaction year.

In [14]:
# Part 1: Getting the years the mrt stations were built. For stations that underwent revelopment
#  to include more lines, the later year was appended
mrt_year_list = []
for mrt in df['mrt_name']:
    if mrt.lower() in ('yio chu kang', 'ang mo kio', 'braddell', 'toa payoh', 'novena', 'orchard', 'somerset', 'dhoby ghaut',
                       'city hall', 'raffles place', 'tanjong pagar'):
        mrt_year_list.append(1987)
    elif mrt.lower() in ('khatib', 'yishun', 'tiong bahru', 'redhill', 'queenstown', 'commonwealth', 'clementi',
                         'jurong east', 'chinese garden', 'lakeside'):
        mrt_year_list.append(1988)
    elif mrt.lower() in ('bugis', 'lavender', 'kallang', 'aljunied', 'paya lebar', 'eunos', 'kembangan', 'bedok', 'tanah merah', 'marina bay',
                         'simei', 'pasir ris'):
        mrt_year_list.append(1989)
    elif mrt.lower() in ('boon lay'):
        mrt_year_list.append(1990)
    elif mrt.lower() in ('admiralty', 'sembawang', 'marsiling', 'kranji', 'yew tee', 'choa chu kang', 'bukit gombak', 'bukit batok'):
        mrt_year_list.append(1996)
    elif mrt.lower() in ('dover'):
        mrt_year_list.append(2001)
    elif mrt.lower() in ('changi airport'):
        mrt_year_list.append(2002)
    elif mrt.lower() in ('clarke quay', 'farrer park', 'boon keng', 'potong pasir',
                         'kovan', 'hougang', 'sengkang', 'punggol'):
        mrt_year_list.append(2003)
    elif mrt.lower() in ('buangkok'):
        mrt_year_list.append(2006)
    elif mrt.lower() in ('bartley', 'marymount', 'lorong chuan', 'bishan', 'serangoon'):
        mrt_year_list.append(2009)
    elif mrt.lower() in ('bras basah', 'esplanade', 'promenade', 'nicoll highway', 'stadium', 'mountbatten', 'dakota', 'tai seng', 
                         'pioneer', 'joo koon'):
        mrt_year_list.append(2010)    
    elif mrt.lower() in ('botanic gardens', 'farrer road', 'holland village', 'one-north', 'kent ridge', 'haw par villa', 
                         'pasir panjang', 'labrador park', 'telok blangah', 'woodleigh', 'buona vista', 'harbourfront'):
        mrt_year_list.append(2011)
    elif mrt.lower() in ('bayfront', 'outram park'):
        mrt_year_list.append(2012)
    elif mrt.lower() in ('downtown', 'telok ayer'):
        mrt_year_list.append(2013)
    elif mrt.lower() in ('rochor', 'stevens', 'tan kah kee', 'sixth avenue', 'king albert park', 'beauty world', 'hillview', 'cashew',
                         'bukit panjang', 'chinatown', 'little india'):
        mrt_year_list.append(2015)
    elif mrt.lower() in ('fort canning', 'bencoolen', 'jalan besar', 'bendemeer', 'geylang bahru', 'mattar', 'ubi', 'kaki bukit',
                         'bedok north', 'bedok reservoir', 'tampines west', 'tampines east', 'upper changi', 'gul circle', 
                         'tuas crescent', 'tuas west road', 'tuas link', 'newton', 'macpherson', 'tampines', 'expo'):
        mrt_year_list.append(2017)
    elif mrt.lower() in ('canberra'):
        mrt_year_list.append(2019)
    elif mrt.lower() in ('woodlands north', 'woodlands south'):
        mrt_year_list.append(2020)
    elif mrt.lower() in ('springleaf', 'lentor', 'mayflower', 'brighthill', 'upper thomson', 'woodlands', 'caldecott'):
        mrt_year_list.append(2021)
    elif mrt.lower() in ('napier', 'orchard boulevard', 'great world', 'havelock', 'maxwell', 'shenton way', 'gardens by the bay'):
        mrt_year_list.append(2022)

df['mrt_built_year'] = mrt_year_list

In [15]:
mrt_development = []
for row in range(0, df.shape[0]):
    if (df['mrt_built_year'][row] - df['Tranc_Year'][row]) > 0: 
        mrt_development.append(5)   # mrt not built yet 
    elif (df['mrt_built_year'][row] - df['Tranc_Year'][row]) >= -2: 
        mrt_development.append(4)   # mrt built but within 2 years of sale
    elif (df['mrt_built_year'][row] - df['Tranc_Year'][row]) >= -5: 
        mrt_development.append(3)   # mrt built but within 5 years of sale
    elif (df['mrt_built_year'][row] - df['Tranc_Year'][row]) >= -7: 
        mrt_development.append(2)   # mrt built but within 7 years of sale
    else:
        mrt_development .append(1)   # the development of the mrt should not matter anymore

df['mrt_development'] = mrt_development


 dist_nearest_top_school: it is the distance in meters to the nearest top school

In [20]:
top_sch_data = pd.read_csv('top_sch_coords.csv', encoding='latin1')
top_sch_data.head()

,Town,School Name,2C Vacancy,2C Subscription,Subscription Rate,Top School,lat,long
0,Ang Mo Kio,CHIJ St. Nicholas GirlsÄô,20,61,305.00%,1,1.373449,103.834078
1,Bedok,Red Swastika,25,77,308.00%,1,1.333641,103.934619
2,Bedok,Temasek,30,95,316.70%,1,1.273404,103.817966
3,Bishan,Ai Tong,20,66,330.00%,1,1.360713,103.833017
4,Bishan,Catholic High,21,48,228.60%,1,1.354857,103.844995


In [23]:
for df_index, df_row in df.iterrows():
    coord_trans = (df_row['Latitude'], df_row['Longitude'])
    dist_list = []
    for top_sch_data_index, top_sch_data_row in top_sch_data.iterrows():
        coord_top_sch1 = (top_sch_data_row['lat'], top_sch_data_row['long'])
        distance_km = geodesic(coord_trans, coord_top_sch1).kilometers
        dist_list.append(distance_km)
    min_dist = min(dist_list)
    df.at[df_index, 'dist_nearest_top_school'] = min_dist

In [24]:
df.head()

,town,flat_type,storey_range,floor_area_sqm,lease_commence_date,resale_price,Tranc_Year,Tranc_Month,mid,hdb_age,...,mrt_nearest_distance,mrt_name,bus_stop_nearest_distance,pri_sch_nearest_distance,pri_sch_name,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,dist_nearest_top_school
0,KALLANG/WHAMPOA,4 ROOM,10 TO 12,90.0,2006,680000.0,2016,5,11,15,...,330.083069,Kallang,29.427395,1138.633422,Geylang Methodist School,78,1,1.317659,103.882504,1.895540
1,BISHAN,5 ROOM,07 TO 09,130.0,1987,665000.0,2012,7,8,34,...,903.659703,Bishan,58.207761,415.607357,Kuo Chuan Presbyterian Primary School,45,1,1.349783,103.854529,0.993119
2,BUKIT BATOK,EXECUTIVE,13 TO 15,144.0,1997,838000.0,2013,7,14,24,...,1334.251197,Bukit Batok,214.747860,498.849039,Keming Primary School,39,0,1.345245,103.756265,1.858299
3,BISHAN,4 ROOM,01 TO 05,103.0,1992,550000.0,2012,4,3,29,...,907.453484,Bishan,43.396521,389.515528,Catholic High School,20,1,1.354789,103.844934,0.378891
4,YISHUN,4 ROOM,01 TO 03,83.0,1987,298000.0,2017,12,2,34,...,412.343032,Khatib,129.422752,401.200584,Naval Base Primary School,74,0,1.416280,103.838798,0.908935


In [25]:
df.to_csv('with_featured_columns.csv')

Featured Engineering done through Excel and/or Excel PivotTables

1. affluent_index: It is basically ranking the average price by town, ranking the floor category and ranking the flat type.
2. amenity_proximity_score: It is the sum of the average distances between the nearest Mall and Hawker
3. transport_proximity_score: It is the sum of the average distances between the nearest MRT and Bus Stop
4. avg_subs_sch: It is the average subscription rate of the schools in the same town
5. num_top_sch: It is the total number of top schools in the town. And top schools is based off if the subsriptions of a school    exceed 200%
6. dist_nearest_top_school: it is the distance in meters to the nearest top school 
7. CPI: Consumer Price Index by Year
8. GDPM: Gross Domestic Product by Quarter
9. MHI: Median Household Income by Quarter

After exporting, more cleaning was done to remove the raw columns that were used to derive the engineered features

The final csv is contained in 'data.csv' 